# Feature Engineering  
## Preparing Customer Data for Segmentation, ML & Tableau

### Objective
The purpose of this notebook is to transform raw customer behavior data into meaningful analytical features that support:
- Customer segmentation
- Churn / retention modeling
- Tableau dashboarding
- Business decision-making

Feature engineering is guided by business logic, not only statistical convenience.


## 1. Load Cleaned Dataset


In [2]:
import pandas as pd
import numpy as np

# Load data (from previous EDA step)
df = pd.read_csv("../data/raw/ecommerce_customer_behavior.csv")

# Rename columns again for safety (idempotent)
df = df.rename(columns={
    "Customer ID": "customer_id",
    "Gender": "gender",
    "Age": "age",
    "City": "city",
    "Membership Type": "membership_type",
    "Total Spend": "total_spend",
    "Items Purchased": "items_purchased",
    "Average Rating": "avg_rating",
    "Discount Applied": "discount_applied",
    "Days Since Last Purchase": "days_since_last_purchase",
    "Satisfaction Level": "satisfaction_level"
})

df.head()


,customer_id,gender,age,city,membership_type,total_spend,items_purchased,avg_rating,discount_applied,days_since_last_purchase,satisfaction_level
0,101,Female,29,New York,Gold,1120.20,14,4.6,True,25,Satisfied
1,102,Male,34,Los Angeles,Silver,780.50,11,4.1,False,18,Neutral
2,103,Female,43,Chicago,Bronze,510.75,9,3.4,True,42,Unsatisfied
3,104,Male,30,San Francisco,Gold,1480.30,19,4.7,False,12,Satisfied
4,105,Male,27,Miami,Silver,720.40,13,4.0,True,55,Unsatisfied


## 2. RFM-Inspired Feature Design

We apply an RFM-style framework adapted to customer-level data:

- **Recency** → Days Since Last Purchase
- **Frequency** → Items Purchased
- **Monetary** → Total Spend

These dimensions are widely used in CRM and retention analytics.


## 3. Spend Segmentation

Customers are grouped into spend tiers to support:
- Segmentation
- Tableau visualization
- High-value customer identification


In [3]:
df["spend_segment"] = pd.qcut(
    df["total_spend"],
    q=3,
    labels=["Low Spend", "Medium Spend", "High Spend"]
)

df[["total_spend", "spend_segment"]].head()


,total_spend,spend_segment
0,1120.20,High Spend
1,780.50,Medium Spend
2,510.75,Low Spend
3,1480.30,High Spend
4,720.40,Medium Spend


## 4. Purchase Frequency Segmentation

Purchase frequency helps distinguish loyal customers from occasional shoppers.


In [4]:
df["frequency_segment"] = pd.qcut(
    df["items_purchased"],
    q=3,
    labels=["Low Frequency", "Medium Frequency", "High Frequency"]
)

df[["items_purchased", "frequency_segment"]].head()


,items_purchased,frequency_segment
0,14,Medium Frequency
1,11,Medium Frequency
2,9,Low Frequency
3,19,High Frequency
4,13,Medium Frequency


## 5. Recency Segmentation (Churn Signal)

Recency is one of the strongest indicators of churn risk.
Customers with long inactivity periods are more likely to churn.


In [6]:
df["recency_segment"] = pd.qcut(
    df["days_since_last_purchase"],
    q=3,
    labels=["Recent", "Moderate", "Inactive"]
)

df[["days_since_last_purchase", "recency_segment"]].head()


,days_since_last_purchase,recency_segment
0,25,Moderate
1,18,Recent
2,42,Inactive
3,12,Recent
4,55,Inactive


## 6. Churn Risk Label

We define a churn-risk proxy based on customer inactivity.
This label will be used as the primary ML target.

Business assumption:
Customers inactive for long periods are at higher risk of churn.


In [7]:
# Define churn threshold (business-driven)
churn_threshold = df["days_since_last_purchase"].quantile(0.66)

df["churn_risk"] = np.where(
    df["days_since_last_purchase"] >= churn_threshold,
    1,  # High churn risk
    0   # Low churn risk
)

df["churn_risk"].value_counts()


churn_risk
0    228
1    122
Name: count, dtype: int64

## 7. Satisfaction Encoding

Satisfaction levels are converted into ordinal values to support modeling.


In [8]:
satisfaction_map = {
    "Unsatisfied": 0,
    "Neutral": 1,
    "Satisfied": 2
}

df["satisfaction_encoded"] = df["satisfaction_level"].map(satisfaction_map)


## 8. Loyalty Score (Composite Feature)

A composite loyalty score combines spending, frequency, and recency into a single metric.
This feature is useful for segmentation and dashboarding.


In [9]:
# Normalize components
df["spend_norm"] = (df["total_spend"] - df["total_spend"].min()) / (df["total_spend"].max() - df["total_spend"].min())
df["freq_norm"] = (df["items_purchased"] - df["items_purchased"].min()) / (df["items_purchased"].max() - df["items_purchased"].min())
df["recency_norm"] = 1 - (
    (df["days_since_last_purchase"] - df["days_since_last_purchase"].min()) /
    (df["days_since_last_purchase"].max() - df["days_since_last_purchase"].min())
)

# Loyalty score
df["loyalty_score"] = (
    0.4 * df["spend_norm"] +
    0.3 * df["freq_norm"] +
    0.3 * df["recency_norm"]
)

df[["loyalty_score"]].describe()


,loyalty_score
count,350.000000
mean,0.478991
std,0.263220
min,0.149430
25%,0.248585
50%,0.457804
75%,0.648038
max,1.000000


## 9. Categorical Encoding

Categorical variables are encoded to prepare the dataset for machine learning models.


In [10]:
df_ml = pd.get_dummies(
    df,
    columns=["gender", "membership_type", "city"],
    drop_first=True
)

df_ml.head()


,customer_id,age,total_spend,items_purchased,avg_rating,discount_applied,days_since_last_purchase,satisfaction_level,spend_segment,frequency_segment,...,recency_norm,loyalty_score,gender_Male,membership_type_Gold,membership_type_Silver,city_Houston,city_Los Angeles,city_Miami,city_New York,city_San Francisco
0,101,29,1120.20,14,4.6,True,25,Satisfied,High Spend,Medium Frequency,...,0.703704,0.616912,False,True,False,False,False,False,True,False
1,102,34,780.50,11,4.1,False,18,Neutral,Medium Spend,Medium Frequency,...,0.833333,0.469024,True,False,True,False,True,False,False,False
2,103,43,510.75,9,3.4,True,42,Unsatisfied,Low Spend,Low Frequency,...,0.388889,0.195565,False,False,False,False,False,False,False,False
3,104,30,1480.30,19,4.7,False,12,Satisfied,High Spend,High Frequency,...,0.944444,0.926125,True,True,False,False,False,False,False,True
4,105,27,720.40,13,4.0,True,55,Unsatisfied,Medium Spend,Medium Frequency,...,0.148148,0.284654,True,False,True,False,False,True,False,False


## 10. Final Feature Review

At this stage, the dataset contains:
- Engineered behavioral segments
- Churn risk label
- Encoded categorical variables
- Composite loyalty metrics

The data is now ready for machine learning and Tableau visualization.


In [11]:
df_ml.to_csv("../data/processed/customer_features_ml.csv", index=False)
